INVESTMENT OPTION

READING DATASET

In [1]:
import pandas as pd
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df=pd.read_excel('Investment_Option.xlsx')

CLEANING THE COLUMN

In [2]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
    .str.replace(r"[^a-z0-9_]", "", regex=True)
)


CHANGING COLUMNS INTO BINARY COLUMNS

In [3]:

df['investment_avenues'] = df['investment_avenues'].str.lower().map({'yes': 1, 'no': 0, 'y': 1, 'n': 0})
df['investment_experience'] = df['investment_experience'].str.lower().map({'yes': 1, 'no': 0, 'y': 1, 'n': 0})

CHANGING EXPECT COLUMN AS AVERAGE VALUE

In [4]:
def extract_midpoint_percent(val):
    if isinstance(val, str):
        match = re.findall(r"(\d+)", val)
        if len(match) == 2:
            return (int(match[0]) + int(match[1])) / 2
    return None
df['expected_return_pct'] = df['expect'].apply(extract_midpoint_percent)


In [5]:
df['investment_period'].unique()

array(['1-3 years', 'More than 5 years', '3-5 years', 'Less than 1 year'],
      dtype=object)

CHANGING INTO ORDINAL VARIABLE

In [6]:
period_map = {
    "Less than 1 year": 6,   # assume 6 months
    "1-3 years": 24,         # midpoint ~2 years
    "3-5 years": 48,         # midpoint ~4 years
    "More than 5 years": 72           # assume 6 years or more
}
df["investment_period_months"] = df["investment_period"].map(period_map)

RE-CHANGING VALUES OF INCOME

In [7]:
mean_income=(350000+400000+550000)/3
df['income'] = df['income'].replace({
    34544: 350000,
    98809885: 400000,
    556449: 550000,
    0:mean_income
})
df['income'] = df['income'].astype(int)



FEATURE ENGINEERING

In [8]:
df.head()

,gender,age,investment_avenues,mutual_funds,equity_market,debentures,government_bonds,fixed_deposits,objective,income,investment_period,invstment_amount,invest_monitor,investment_experience,expect,expected_return_pct,investment_period_months
0,Female,34,1,1,2,5,3,7,Capital Appreciation,100000,1-3 years,1000,Monthly,1,20%-30%,25.0,24
1,Female,23,1,4,3,2,1,5,Capital Appreciation,100000,More than 5 years,1000,Weekly,1,20%-30%,25.0,72
2,Male,30,1,3,6,4,2,5,Capital Appreciation,100000,3-5 years,1000,Daily,1,20%-30%,25.0,48
3,Male,22,1,2,1,3,7,6,Income,100000,Less than 1 year,1000,Daily,1,10%-20%,15.0,6
4,Female,24,0,2,1,3,6,4,Income,100000,Less than 1 year,1000,Daily,1,20%-30%,25.0,6


Investment_to_income_ratio

In [9]:
df = df.rename(columns={"invstment_amount": "investment_amount"})
df['investment_to_income_ratio'] = df['investment_amount'] / df['income']
#Normalize investment size across income levels.

Risk_score

In [10]:
risk_columns = ['mutual_funds', 'equity_market', 'debentures', 'government_bonds', 'fixed_deposits']
df['risk_score'] = df[risk_columns].mean(axis=1)  # (average of asset preferences)


Liquidity_preference

In [11]:
df['liquidity_preference'] = df['fixed_deposits'] - df['equity_market']   # (fixed income vs market risk)

Age_bucket

In [12]:
df['age_bucket'] = pd.cut(
    df['age'],
    bins=[18, 25, 30, 35, 40, 50, 100],
    labels=['18-25', '26-30', '31-35', '36-40', '41-50', '51+'],
    right=False
)
#(grouping into stages of life)

Objective_encoded 

In [13]:
objective_map = {
    'Income': 0,
    'Capital Appreciation': 1,
    'Growth': 2
}
df['objective_encoded'] = df['objective'].map(objective_map)

#(convert goal text to numbers)

MODEL PREPARATION

INPUTS FOR CLASSIFICATION

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['age_bucket_encoded'] = le.fit_transform(df['age_bucket'])


In [15]:
classification_features = [
    'income',
    'investment_amount',
    'investment_to_income_ratio',
    'risk_score',
    'liquidity_preference',
    'investment_experience',
    'age_bucket_encoded'  # Convert to dummy
]

INPUTS FOR REGRESSION

In [16]:
regression_features = classification_features  # same as above


Encode Categorical Columns

SPLITTING DATASET

In [17]:
from sklearn.model_selection import train_test_split

# For classification
X_class = df[classification_features]
y_class = df['objective_encoded']  # or correct column name

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42
)

# For regression
X_reg = df[regression_features]
y_reg = df['expected_return_pct']  # replace with actual column

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)


MODEL BUILDING

CLASSIFICATION

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize model
clf = DecisionTreeClassifier(random_state=42)

# Fit the model
clf.fit(X_train_c, y_train_c)

# Predict on test set
y_pred_c = clf.predict(X_test_c)

# Evaluate
print("Accuracy:", accuracy_score(y_test_c, y_pred_c))
print(classification_report(y_test_c, y_pred_c))



Accuracy: 0.9091796875
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        82
           1       0.96      0.90      0.93       673
           2       0.78      0.91      0.84       269

    accuracy                           0.91      1024
   macro avg       0.91      0.94      0.92      1024
weighted avg       0.92      0.91      0.91      1024



REGRESSION

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Train/test split
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

# Model
reg = LinearRegression()
reg.fit(X_train_r, y_train_r)

# Predict
y_pred_r = reg.predict(X_test_r)

# Evaluate
print("MSE:", mean_squared_error(y_test_r, y_pred_r))
print("R2 Score:", r2_score(y_test_r, y_pred_r))


MSE: 15.713207846111828
R2 Score: 0.2110017511806579


In [20]:
df.to_excel("updated_dataset.xlsx", index=False)


PICKLE FILE 

In [21]:
import joblib

joblib.dump(clf, 'classification_model.pkl')
joblib.dump(reg, 'regression_model.pkl')


['regression_model.pkl']